In [1]:
def lighten_color(color, amount=0.5):
    """
    source: https://stackoverflow.com/questions/37765197/darken-or-lighten-a-color-in-matplotlib
    
    Lightens the given color by multiplying (1-luminosity) by the given amount.
    Input can be matplotlib color string, hex string, or RGB tuple.

    Examples:
    >> lighten_color('g', 0.3)
    >> lighten_color('#F034A3', 0.6)
    >> lighten_color((.3,.55,.1), 0.5)
    """
    import matplotlib.colors as mc
    import colorsys
    
    try:
        c = mc.cnames[color]
    except:
        c = color
    c = colorsys.rgb_to_hls(*mc.to_rgb(c))
    
    return colorsys.hls_to_rgb(c[0], 1 - amount * (1 - c[1]), c[2])

In [2]:
def plot_conductivities(graph,tdens_stack, tdens_stack_f, tdens_opt_f, time_step):

    import matplotlib.pyplot as plt
    import matplotlib
    from matplotlib.lines import Line2D
    import numpy as np

    size = 3
    fig, ax = plt.subplots(figsize=(18/size,12/size))
    ax.margins(x=0.0)

    # layout - 1
    cmap = matplotlib.cm.get_cmap('viridis_r')
    c_list = [lighten_color(cmap(i/graph.number_of_edges()), 0.9) for i in range(graph.number_of_edges())]
    c_list_light = [lighten_color(cmap(i/graph.number_of_edges()), 1.1) for i in range(graph.number_of_edges())]


    # plot
    index_ = np.argsort(tdens_stack[-1,:])
    n = 0
    for i, tdens_edge in zip(index_, tdens_opt_f):
        plt.plot(np.arange(tdens_stack[:301,i].size)*time_step, tdens_stack[:301,index_[i]], color=c_list[i], linestyle="solid", marker='o', ms=0, lw=1.5, alpha=1, zorder=n)
        plt.plot(np.arange(tdens_stack[:301+20,i].size)*time_step, tdens_stack[:301+20,index_[i]], color=c_list[i], linestyle=":", marker='o', ms=0, lw=1.5, alpha=1, zorder=n)
        plt.plot(400*time_step + np.arange(tdens_stack[:301,i].size)*time_step, tdens_stack[-301:,index_[i]], color=c_list[i], linestyle="solid", marker='o', ms=0, lw=1.5, alpha=1, zorder=n)
        plt.plot((400-20)*time_step + np.arange(tdens_stack[:301+20,i].size)*time_step, tdens_stack[-301-20:,index_[i]], color=c_list[i], linestyle=":", marker='o', ms=0, lw=1.5, alpha=1, zorder=n)
        plt.hlines(tdens_stack_f[-1,index_[i]], 400*time_step, 400*time_step + tdens_stack[:301,i].size*time_step, color=c_list_light[i], linestyle="--", lw=1, alpha=0.7, zorder=i)
        n=n+1

    # layout - 2
    my_xticks = ['0','',r'$3\cdot 10^2$',r"$\,\,\,\, \approx 10^4$","",""]
    plt.xticks([0,1.5,3,4,5.5,7],my_xticks,fontsize=16)
    plt.yticks(fontsize=16)
    plt.xlabel(r"iterations", fontsize=16)
    plt.ylabel(r"conductivities", fontsize=16)
    ylim_ = np.max(tdens_stack) + np.max(tdens_stack)/10
    plt.ylim(-1,ylim_)

    plt.text(1, ylim_ + np.max(tdens_stack)/10, r"$t < t_\mathrm{STAB}$", size=16)
    plt.text(5, ylim_ + np.max(tdens_stack)/10, r"$t \gg t_\mathrm{STAB}$", size=16)

    # legend
    custom_lines = [Line2D([0], [0], color="grey", lw=2),
                    Line2D([0], [0], color="grey", ls="dashed", lw=1)]
    ax.legend(custom_lines, [r'$\mu_e(t)$', r'$\bar{\mu}_e^\infty$'], fontsize=14, framealpha=0,
                            edgecolor="white", fancybox=False, labelspacing=0.0, borderpad=0.1, borderaxespad=0.2)

    plt.show()

In [3]:
def plot_functionals(tdens_stack_f, cost_stack_f, tdens_stack, flux_stack,
                                length, time_step, time_step_f, pflux):

    import matplotlib.pyplot as plt
    import numpy as np
    from matplotlib.lines import Line2D

    size = 3
    fig, ax = plt.subplots(figsize=(18/size,12/size))
    ax.margins(x=0.0)

    # running average of the Lyapunov
    N=int(1/time_step)
    c_time = []
    for flux, tdens in zip(flux_stack, tdens_stack):
        c = np.sum((length/tdens) * (np.abs(flux)**2)) + 1/(2-pflux)*np.sum(length*tdens**(2-pflux))
        c_time.append(c)  
    c_time = np.array(c_time)
    running_cost = np.convolve(c_time, np.ones(N)/N, mode='valid')

    # plot
    plt.plot(np.arange(running_cost.size)*time_step, running_cost, lw=3)
    plt.plot(np.arange(tdens_stack_f[:,0].size)*time_step_f, cost_stack_f, lw=3, ls="--")

    # layout
    plt.xscale("log")
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.xlabel(r"$t$", fontsize=16)
    plt.ylabel(r"Lyapunov", fontsize=16)

    # legend
    custom_lines = [Line2D([0], [0], color="C0", lw=3),
                    Line2D([0], [0], color="C1", ls="--", lw=3),
                    Line2D([0], [0], color="C2", ls=":", lw=3)]
    ax.legend(custom_lines, [r'$\langle \mathcal{L}_\gamma \rangle_T $', r'$\bar{\mathcal{L}}_\gamma$'], fontsize=14, framealpha=0,
                            edgecolor="grey", fancybox=False, labelspacing=0.1, borderaxespad=0.2)


    plt.show()

In [4]:
def plot_network(graph, tdens_stack, tdens_opt_f, forcing):
    
    !pip install -q osmnx
    import osmnx as ox
    import matplotlib.pyplot as plt
    import numpy as np
    import matplotlib
    import networkx as nx
    
    # layout - 1
    cmap = matplotlib.cm.get_cmap('viridis_r')
    c_list = [lighten_color(cmap(i/graph.number_of_edges()), 0.9) for i in range(graph.number_of_edges())]
    c_list_light = [lighten_color(cmap(i/graph.number_of_edges()), 1.1) for i in range(graph.number_of_edges())]
    
    # loading Bordeaux data
    print("Loading geospatial data from OpenStreetMap, this may take a while...")
    G = ox.graph_from_address('Bordeaux, France', network_type='drive',  dist=3000, simplify=True, clean_periphery=True)

    # layout - 2
    size = 2
    fig, ax = plt.subplots(figsize=(14/size,14/size))
    prop_coeff = 2.

    keys = [i for i in dict(graph.nodes.data()).keys()]
    values = [i["pos"] for i in dict(graph.nodes.data()).values()]
    pos = {k : v for k,v in zip(keys,values)}
    time_range = np.arange(0,100,1)
    index_ = np.argsort(tdens_stack[-1,:])
    c_list_order = np.array(c_list)[index_]

    # plot
    nx.draw_networkx_nodes(graph, pos=pos, ax=ax, nodelist=np.arange(graph.number_of_nodes()), node_color="gray", node_size=np.max(np.abs(forcing(time_range)), axis=1))
    nx.draw_networkx_edges(graph, pos=pos, ax=ax, edge_color=prop_coeff*tdens_opt_f**0.5, width=prop_coeff*tdens_opt_f**0.5)
    fig_1, ax_1 = ox.plot.plot_graph(G, ax=ax, node_zorder=-3, edge_color="silver", edge_alpha=0.3, save=True)

    plt.show()
